In [ ]:
import pandas as pd
import joblib
pd.set_option('max_colwidth', 500)
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams.update({'font.size': 9})
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression, Lasso
from utils import *
import xgboost as xgb
%matplotlib notebook

%load_ext autoreload
%autoreload 2
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))

# 0 - Problem Statement
The goal of this notebook is to show possible usage of Random Forest regressor for timeseries forecasting; in the notebook multiple example are given with the usage of a general purpose library.<br>
The notebook contains the following sections:

1. Introduction to univariate & multivariate timeseries datasets
2. Cross validation: two different techniques
3. CART regressor on univariate timeseries
4. CART on multivariate timeseries

# 1 - Introduction to univariate & multivariate timeseries datasets
A timeseries without covariates is an "univariate" timeseries.<br>
Here below an example of female births dataset, that is the monthly births across three years. <br>
Credits: <br>
https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-total-female-births.csv <br>
https://machinelearningmastery.com/random-forest-for-time-series-forecasting/

In [ ]:
filename = 'daily-total-female-births.csv'
# load the dataset
series = pd.read_csv(filename, header=0, index_col=0, parse_dates=True)
values = series.values

# plot dataset
ax = series.plot(style='.-')
ax.grid()
plt.show(block=False)

A timeseries with covariates is a multivaraite timeseries.<br>
This dataset contains 19 different features such as air temperature, atmospheric pressure, and humidity collected from 2009 to 2016 with 1 records every hour.<br>
Credits: <br>https://www.bgc-jena.mpg.de/wetter/ <br>https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip <br>
https://www.tensorflow.org/tutorials/structured_data/time_series

In [ ]:
filename = 'weather_dataset.csv'
# load the dataset
data = pd.read_csv(filename, header=0, index_col=0, parse_dates=True)

# plot some features
feat = ['T (degC)', 'p (mbar)', 'rho (g/m**3)', 'T (degC)']
data.plot(y=feat,style='-',grid=True,subplots=True)
plt.show(block=False)

The models in this notebook will make a set of predictions on both datasets based on a window of consecutive samples. <br>

The main features of the input windows are:

- The width (number of time steps) of the input and label windows. (parameter "n_in")
- The time offset between them. (parameter "n_ahead")
- Single-time-step and multi-time-step predictions. (parameter "single_output")
- Single-output or multi-output predictions. (for multivariate only)

This notebook will use the function "series_to_supervised" to do the data windowing.<br>


<img src="figures/TS_exaplained_pw.png">

Here below an example of the usage of the "series_to_supervised" function.<br>
Timestamps till <b>t-1</b> are the training samples.

In [ ]:
############## Params START ##############
n_in = 6 # Number of "previous data" to use as input
n_ahead = 4 # Number of step ahead to predict
single_output = False # If True, predicts only the last n_ahead value; else it predicts all the values "in between"
n_out = 1 if single_output else n_ahead # Number of outputs of the model
filename = 'daily-total-female-births.csv'
############## Params End ##############

# load the dataset
series = pd.read_csv(filename, header=0, index_col=0)
values = series.values

# transform the time series data into supervised learning
data = series_to_supervised(values, n_in=n_in, n_out=n_ahead, single_output=single_output)
data.head()

In [ ]:
############## Params START ##############
n_in = 2 # Number of "previous data" to use as input
n_ahead = 2 # Number of step ahead to predict
single_output = True # If True, predicts only the last n_ahead value; else it predicts all the values "in between"
n_out = 1 if single_output else n_ahead # Number of outputs of the model
filename = 'weather_dataset.csv'
############## Params End ##############

# load the dataset
df = pd.read_csv(filename, header=0, index_col=0)
df = df[['T (degC)', 'p (mbar)', 'rho (g/m**3)', 'T (degC)']]

# transform the time series data into supervised learning
data = series_to_supervised(df.values, n_in=n_in, n_out=n_ahead, single_output=single_output)
data.head()

# 2 - Cross validation: two different techniques
The classical k-fold cross validation cannot be used for timeseries problems because the "shuffling" of the records can causa data leakage: information from the future will "leak" into the current prediction.<br>
For this reason, two different cross-validation techniques are applied:

- Sliding vs Expanding windows
- Walk forward cross validation

## 2.1 Sliding vs Examping window
The difference between sliding and expanding windows it is in the train dataset dimension: in the sliding window it is fixed, in the expanding window increases with the training. For both techniques the number of splits are specified by the user. <br>
Figures credits: https://www.analyticsvidhya.com/blog/2021/06/random-forest-for-time-series-forecasting/

<img src="figures/SlidingExpanding.png">

In this notebook only <b>expanding window</b> will be used since it uses most of the data for the training. <br>
Here below an example of the expanding window approach: <br>
Dataset credits: https://machinelearningmastery.com/backtest-machine-learning-models-time-series-forecasting/

In [ ]:
series = pd.read_csv('sunspots.csv', header=0, index_col=0)
X = series.values
splits = TimeSeriesSplit(n_splits=3)
plt.figure()
index = 1
for train_index, test_index in splits.split(X):
    train = X[train_index][:,0]
    test = X[test_index][:,0]
    print('Observations: %d' % (len(train) + len(test)))
    print('Training Observations: %d' % (len(train)))
    print('Testing Observations: %d' % (len(test)))
    plt.subplot(3,1,0 + index)
    plt.plot(train)
    plt.plot([None for i in train] + [x for x in test])
    plt.grid()
    index += 1
plt.show()

## 2.2 Walk forawd cross validation
Walk forward cross validation is a particular case of expanding window where the dimension of the test set is of one record only, so the number of splits will be equal to the number of test sample used for cross validation (parameter "n_test" for function "walk_forward_validation").

<img src="figures/splits.png">

# 3 - CART regressor on univariate timeseries
The main goal of this notebook is to use CART (Classification And Regression Tree) methods for timeseries forecasting. <br>
In this section multiple example of usage are presented and, each time a new CART methodology is used, the main parameter are described.<br>

<b>NOTE:</b> Two naive models are used for performance comparison:
- The persistence model which simply predicts based on last seen record.
- The mean model which simply predicts the average of the training samples.

<b>NOTE:</b> the metric used to compare the model are the Mean Absolute Error (MAE) and the Root Mean Square Error (RMSE).

## 3.1 Univariate Analysis
In this section three differnt CART techniques will be described and applied to the univariate case. This case is trivial but it will be used as a reference case for each technique.

In [ ]:
output_metrics = pd.DataFrame(data={'Persistence':[float('nan'),float('nan')],
                                    'Mean':[float('nan'),float('nan')],
                                    'DecisionTree':[float('nan'),float('nan')],
                                    'RandomForest':[float('nan'),float('nan')],
                                    'GradientBoosting':[float('nan'),float('nan')],
                                    'SuperLearner':[float('nan'),float('nan')]},index=['MAE','RMSE'])

### 3.1.1 - Decision Tree Regressor
The Decision Tree Regressor is the first CART used in this notebook. The main parameters of this CART are the following:
- "criterion": the greedy algorithm based on the CART training process, need a function to decide the best split. The criterion specifies which function to be used.
- "max_depth": maximum depth of the tree. This parameter controls the maximum depth of the tree to avoid overfitting.
- "min_samples_split": The minimum number of samples required to split a node.
- "min_samples_leaf": The minimum number of samples required to be at a leaf node.
- "ccp_alpha": The tree can be pruned after its growth. To prune the tree a cost associated to the complexity is added in the cost function of the trainig; this parameter regulates the cost associated to the complexity.

Here below there is example of decision tree usage on univariate timeseries (the female dataset births dataset).

In [ ]:
############## Data Preprocessing Params ##############
n_in = 6 # Number of "previous data" to use as input
n_ahead = 1 # Number of step ahead to predict
single_output = True # If True, predicts only the last n_ahead value; else it predicts all the values "in between"
n_out = 1 if single_output else n_ahead # Number of outputs of the model
train_size = 0.8 # Size of the training set
n_splits = 5 # Number of splits for cross validation
filename = 'daily-total-female-births.csv'
############## Data Preprocessing Params ##############

# load the dataset
series = pd.read_csv(filename, header=0, index_col=0)

# Transform the time series data into supervised learning
data = series_to_supervised(series.values, n_in=n_in, n_out=n_ahead, single_output=single_output)
data_train = data.values[0:int(train_size*data.shape[0])]
data_test = data.values[int(train_size*data.shape[0]):]
testX, testy = data_test[:, :-n_out], data_test[:, -n_out:]
                            
# Start the training process
predictions = list()
predictions_persistence = list()
predictions_mean = list()
y = list()
# Split dataset
ts_splits = TimeSeriesSplit(n_splits=n_splits)
# Step over each split
for train_index, val_index in ts_splits.split(data_train):
    train = data_train[train_index]
    val = data_train[val_index]
    
    # Split test row into input and output columns
    valX, valy = val[:, :-n_out], val[:, -n_out:]
    
    # transform list into array
    train = np.asarray(train)
    
    # split into input and output columns
    trainX, trainy = train[:, :-n_out], train[:, -n_out:]
    
    # fit model
    model = DecisionTreeRegressor(random_state=0)
    model.fit(trainX, trainy.flatten())
    
    # Perform prediction
    yhat = model.predict(valX)
    
    # Use persistence & mean models
    predictions_persistence.append(valX[:,-1])
    predictions_mean.append(valX.mean(axis=1))
    
    # store forecast in list of predictions
    predictions.append(yhat.flatten())
    y.append(valy.flatten())

# Refit the model on the entire training dataset
model = DecisionTreeRegressor(random_state=0)
model.fit(data_train[:, :-n_out], data_train[:, -n_out:])

# Estimate prediction error
predictions = np.array(predictions).flatten()
predictions_persistence = np.array(predictions_persistence).flatten()
predictions_mean = np.array(predictions_mean).flatten()
y = np.array(y).flatten()
print('Decision Tree - training MAE = {:.3f}'.format(mean_absolute_error(y, predictions)))
print('Persistence - training MAE = {:.3f}'.format(mean_absolute_error(y, predictions_persistence)))
print('Mean - training MAE = {:.3f}'.format(mean_absolute_error(y, predictions_mean)))
# Evaluate regressor on test set
print('Decision Tree - test MAE = {:.3f}'.format(mean_absolute_error(testy, model.predict(testX))))
print('Persistence - test MAE = {:.3f}'.format(mean_absolute_error(testy, testX[:,-1])))
print('Mean - test MAE = {:.3f}'.format(mean_absolute_error(testy, testX.mean(axis=1))))
print('Decision Tree - test RMSE = {:.3f}'.format(rmse(testy, model.predict(testX))))
print('Persistence - test RMSE = {:.3f}'.format(rmse(testy, testX[:,-1])))
print('Mean - test RMSE = {:.3f}'.format(rmse(testy, testX.mean(axis=1))))
output_metrics.loc['MAE','Persistence'] = mean_absolute_error(testy, testX[:,-1])
output_metrics.loc['MAE','Mean'] = mean_absolute_error(testy, testX.mean(axis=1))
output_metrics.loc['RMSE','Persistence'] = rmse(testy, testX[:,-1])
output_metrics.loc['RMSE','Mean'] = rmse(testy, testX.mean(axis=1))

The same training process can be done by using Scikit-learn package <b>GridSearchCV</b>.

In [ ]:
gs = GridSearchCV(estimator=DecisionTreeRegressor(random_state=0), cv=ts_splits, param_grid={}, scoring='neg_mean_absolute_error')
gs.fit(data_train[:, :-n_out], data_train[:, -n_out:])
print('Decision Tree - training MAE = {:.3f}'.format(abs(gs.cv_results_['mean_test_score'][0])))
print('Decision Tree - test MAE = {:.3f}'.format(mean_absolute_error(testy, gs.best_estimator_.predict(testX))))

The <b>GridSearchCV</b> can be also used for <b>hyper-parameters</b> tuning.

In [ ]:
force_retraining = False
gs = GridSearchCV(estimator=DecisionTreeRegressor(random_state=0), cv=ts_splits, 
                  param_grid={
                      'criterion':['mae','mse'],
                      'max_depth':[3,10,100,300],
                      'min_samples_split': [5,10,100,1000],
                      'ccp_alpha': [0,0.0001,0.001,0.01,1]                      
                  },
                  scoring='neg_mean_absolute_error',
                  verbose = 0, n_jobs=-1)
if force_retraining:
    gs.fit(data_train[:, :-n_out], data_train[:, -n_out:])
    joblib.dump(gs, 'models/Univariate/DecisionTree.pkl')
else:
    gs = joblib.load('models/Univariate/DecisionTree.pkl')
print('Decision Tree - training MAE = {:.3f}'.format(abs(gs.cv_results_['mean_test_score'][0])))
print('Decision Tree - test MAE = {:.3f}'.format(mean_absolute_error(testy, gs.best_estimator_.predict(testX))))
print('Decision Tree - test RMSE = {:.3f}'.format(rmse(testy, gs.best_estimator_.predict(testX))))
gs_res = pd.DataFrame(gs.cv_results_).sort_values(by='rank_test_score')[['params','mean_test_score']]
gs_res.head(10)

The best model of the above list is the number 2, since it is the "simplest" one among the group with best performances.

In [ ]:
idx = 2
dt_tuned = DecisionTreeRegressor(random_state=0)
dt_tuned.set_params(**gs_res.loc[idx,'params'])
dt_tuned.fit(data_train[:, :-n_out], data_train[:, -n_out:])
yhat = dt_tuned.predict(testX)
def plot_univariate(yhat,label):
    plt.figure(figsize=(9,5))
    plt.plot(testy,'.-',label='Data')
    plt.plot(yhat,'.-',label=label)
    plt.plot(testX[:,-1],'.-',label='Persistence model', alpha=0.3)
    plt.plot(testX.mean(axis=1),'.-',label='Mean model',alpha=0.3)
    plt.legend(frameon=False, loc=0)
    plt.grid()
plot_univariate(yhat,label='Decision Tree')
output_metrics.loc['MAE','DecisionTree'] = mean_absolute_error(testy, dt_tuned.predict(testX))
output_metrics.loc['RMSE','DecisionTree'] = rmse(testy, dt_tuned.predict(testX))

### 3.1.2 - Random Forest Regressor
The Random Forest Regressor is the second CART used in this notebook. This method combines an ensemble of DecisionTree and weight their output to generate the prediction. This technique allows the applciation a bootstrap operation on the data and the usage of only a portion of the features for each tree; both these features have the goal to mitigate the overfitting. <br>The main parameters of this CART are the following:
- "n_estimator": number of estimator to be trained by the algorithm.
- "criterion": the greedy algorithm based on the CART training process, need a function to decide the best split. The criterion specifies which function to be used.
- "max_depth": maximum depth of the tree. This parameter controls the maximum depth of the tree to avoid overfitting.
- "min_samples_split": The minimum number of samples required to split a node.
- "min_samples_leaf": The minimum number of samples required to be at a leaf node.
- "ccp_alpha": The tree can be pruned after its growth. To prune the tree a cost associated to the complexity is added in the cost function of the trainig; this parameter regulates the cost associated to the complexity.
- "bootstrap": if True, bootstrap procedure will be applied (always set to True in these examples)
- "max_features": The number of features to consider when looking for the best split. 

In [ ]:
force_retraining = False
gs = GridSearchCV(estimator=RandomForestRegressor(random_state=0), cv=ts_splits,
                  param_grid={
                      'n_estimators': [10, 100, 1000],
                      'criterion':['mae'],
                      'max_depth':[10,100],
                      'min_samples_split': [5, 10, 100, 500],
                      'ccp_alpha': [0,0.001]
                  },
                  scoring='neg_mean_absolute_error',
                  verbose=0, n_jobs=-1)
if force_retraining:
    gs.fit(data_train[:, :-n_out], data_train[:, -n_out:].ravel())
    joblib.dump(gs, 'models/Univariate/RandomForest.pkl')
else:
    gs = joblib.load('models/Univariate/RandomForest.pkl')
print('Random Forest - training MAE = {:.3f}'.format(abs(gs.cv_results_['mean_test_score'][0])))
print('Random Forest - test MAE = {:.3f}'.format(mean_absolute_error(testy, gs.best_estimator_.predict(testX))))
gs_res = pd.DataFrame(gs.cv_results_).sort_values(by='rank_test_score')[['params','mean_test_score']]
gs_res.head(10)

The best model of the above list is the number 18, since it is the "simplest" one among the group with best performances.

In [ ]:
idx = 18
rf_tuned = RandomForestRegressor(random_state=0, n_jobs=-1)
rf_tuned.set_params(**gs_res.loc[idx,'params'])
rf_tuned.fit(data_train[:, :-n_out], data_train[:, -n_out:].ravel())
yhat = rf_tuned.predict(testX)
plot_univariate(yhat,label='Random Forest')
output_metrics.loc['MAE','RandomForest'] = mean_absolute_error(testy, rf_tuned.predict(testX))
output_metrics.loc['RMSE','RandomForest'] = rmse(testy, rf_tuned.predict(testX))

### 3.1.3 Gradient Boosting Regressor

The Gradient Boosting Regressor is the third CART used in this notebook. This technique builds an additive model and it allows for the optimization of arbitrary differentiable loss functions. In each stage a regression tree is fit on the negative gradient of the given loss function.<br> The main parameters of this CART are the following:
- "loss": the loss function to be optimized.
- "learning_rate": the learning rate multiplier for each additional tree.
- "subsample": the fraction of samples to be used for fitting the individual base learners. 
- "n_estimator": number of estimator to be trained by the algorithm.
- "criterion": the greedy algorithm based on the CART training process, need a function to decide the best split. The criterion specifies which function to be used.
- "max_depth": maximum depth of the tree. This parameter controls the maximum depth of the tree to avoid overfitting.
- "min_samples_split": The minimum number of samples required to split a node.
- "min_samples_leaf": The minimum number of samples required to be at a leaf node.
- "ccp_alpha": The tree can be pruned after its growth. To prune the tree a cost associated to the complexity is added in the cost function of the trainig; this parameter regulates the cost associated to the complexity.
- "bootstrap": if True, bootstrap procedure will be applied (always set to True in these examples)
- "max_features": The number of features to consider when looking for the best split.
- "validation_fraction" and "n_iter_no_change" and "tol": Parameters for the early stopping criterion (use validation dataset to evaluate the loss and if it does not decrease stop the training).

In [ ]:
force_retraining = False
gs = GridSearchCV(estimator=GradientBoostingRegressor(random_state=0),cv=ts_splits,
                  param_grid={
                      'loss': ['ls','lad'],
                      'n_estimators': [10, 100, 1000],
                      'criterion':['mse'],
                      'max_depth':[5,10,100],
                      'min_samples_split': [5, 10, 30],
                      'learning_rate': [0.000001, 0.0001, 0.01, 0.1]
                  },
                  scoring='neg_mean_absolute_error',
                  verbose=10,n_jobs=-1)
if force_retraining:
    gs.fit(data_train[:, :-n_out], data_train[:, -n_out:].ravel())
    joblib.dump(gs, 'models/Univariate/GradientBoosting.pkl')
else:
    gs = joblib.load('models/Univariate/GradientBoosting.pkl')
print('Gradient Boosting - training MAE = {:.3f}'.format(abs(gs.cv_results_['mean_test_score'][0])))
print('Gradient Boosting - test MAE = {:.3f}'.format(mean_absolute_error(testy, gs.best_estimator_.predict(testX))))
gs_res = pd.DataFrame(gs.cv_results_).sort_values(by='rank_test_score')[['params','mean_test_score']]
gs_res.head(10)

In [ ]:
idx = 9
gb_tuned = GradientBoostingRegressor(random_state=0)
gb_tuned.set_params(**gs_res.loc[idx,'params'])
gb_tuned.fit(data_train[:, :-n_out], data_train[:, -n_out:].ravel())
yhat = gb_tuned.predict(testX)
plot_univariate(yhat,label='Gradient Boosting')
output_metrics.loc['MAE','GradientBoosting'] = mean_absolute_error(testy, gb_tuned.predict(testX))
output_metrics.loc['RMSE','GradientBoosting'] = rmse(testy, gb_tuned.predict(testX))

Very detailed hyper-parameter tuning has been performed on Gradient Boosting algorithm but without success. This CART predicts always a fixed value so suffers of <b>high bias</b> and <b>low variance</b>.

### 3.1.4 SuperLearner
The idea behind the SuperLearner is to <b>combine different regressors</b> and average all the outputs to obtain predicted values. This type of regressor can be useful for a set of well performing models in order to balance out their individual weaknesses. <br>
It has been decided to use the following regressors:

- Decision Tree
- Random Forest
- Linear Regression

The Gradient Boosting algorithm was not sued because of its poor performances.<br>
Another interesting technique is the <b>stacked generalization</b>. This methodology uses a set of models in parallel (SuperLearner) and the output is used as input for a final estimator. This technique is out of scope of this notebook.


In [ ]:
force_retraining = False
reg1 = dt_tuned
reg2 = rf_tuned
reg3 = LinearRegression()
ereg = VotingRegressor(estimators=[('dt', reg1), ('rf', reg2), ('lr', reg3)])
if force_retraining:
    ereg = ereg.fit(data_train[:, :-n_out], data_train[:, -n_out:].ravel())
    joblib.dump(ereg, 'models/Univariate/SuperLearner.pkl')
else:
    ereg = joblib.load('models/Univariate/SuperLearner.pkl')

yhat = ereg.predict(testX)
plot_univariate(yhat,label='SuperLearner')
output_metrics.loc['MAE','SuperLearner'] = mean_absolute_error(testy, ereg.predict(testX))
output_metrics.loc['RMSE','SuperLearner'] = rmse(testy, ereg.predict(testX))

Finally, the performances of all models are compared together on the <b>entire set</b> (training and test). <br>

In [ ]:
plt.figure(figsize=(9,5))
plt.plot(data.values[:,-n_out:],'.-',label='Data', alpha = 0.5)
plt.plot(dt_tuned.predict(data.values[:,:-n_out]),'.-',label='DecisionTree', alpha = 0.5)
plt.plot(rf_tuned.predict(data.values[:,:-n_out]),'.-',label='RandomForest', alpha =0.5)
plt.plot(ereg.predict(data.values[:,:-n_out]),'.-',label='SuperLearner', alpha = 0.5)
plt.legend(frameon=False, loc=0)
plt.grid()
output_metrics

### 3.2 Data Preprocessing Params Tuning
All the regressor shown in the 3.1 sections have the data preprocessing parameter fixed.<br>
In this section different "n_in" (number of "lags") values will be used to generate the prediction.

In [ ]:
force_retraining = False
############## Data Preprocessing Params ##############
n_in_list = [3,6,12,24] # Number of "previous data" to use as input
n_ahead = 1 # Number of step ahead to predict
single_output = True # If True, predicts only the last n_ahead value; else it predicts all the values "in between"
n_out = 1 if single_output else n_ahead # Number of outputs of the model
train_size = 0.8 # Size of the training set
n_splits = 3 # Number of splits for cross validation
filename = 'daily-total-female-births.csv'
############## Data Preprocessing Params ##############

output_metrics_preproc = pd.DataFrame(data={'Persistence - 3':[float('nan'),float('nan')]},index=['MAE','RMSE'])

for n_in in n_in_list:
    print('Testing {}'.format(n_in))
    # load the dataset
    series = pd.read_csv(filename, header=0, index_col=0)

    # Transform the time series data into supervised learning
    data = series_to_supervised(series.values, n_in=n_in, n_out=n_ahead, single_output=single_output)
    data_train = data.values[0:int(train_size*data.shape[0])]
    data_test = data.values[int(train_size*data.shape[0]):]
    testX, testy = data_test[:, :-n_out], data_test[:, -n_out:]
    
    # Naive regressors
    output_metrics_preproc.loc['MAE','Persistence - {}'.format(n_in)] = mean_absolute_error(testy, testX[:,-1])
    output_metrics_preproc.loc['RMSE', 'Persistence - {}'.format(n_in)] = rmse(testy, testX[:,-1])
    output_metrics_preproc.loc['MAE','Mean - {}'.format(n_in)] = mean_absolute_error(testy, testX.mean(axis=1))
    output_metrics_preproc.loc['RMSE','Mean - {}'.format(n_in)] = rmse(testy, testX.mean(axis=1))
    
    # Decision Tree
    gs_dt = GridSearchCV(estimator=DecisionTreeRegressor(random_state=0), cv=ts_splits, 
                  param_grid={'criterion':['mae','mse'],
                              'max_depth':[3,10,100,300],
                              'min_samples_split': [5,10,100,1000],
                              'ccp_alpha': [0,0.0001,0.001,0.01,1]
                             },
                  scoring='neg_mean_absolute_error',
                  verbose = 0, n_jobs=-1)
    if force_retraining:
        gs_dt.fit(data_train[:, :-n_out], data_train[:, -n_out:])
        joblib.dump(gs_dt, 'models/Univariate/n_in_tuning/DecisionTree_nin_{}.pkl'.format(n_in))
    else:
        gs_dt = joblib.load('models/Univariate/n_in_tuning/DecisionTree_nin_{}.pkl'.format(n_in))
    print('DecisionTree on {} tuned!'.format(n_in))    
    yhat = gs_dt.best_estimator_.predict(testX)
    output_metrics_preproc.loc['MAE','Decision Tree - {}'.format(n_in)] = mean_absolute_error(testy, yhat)
    output_metrics_preproc.loc['RMSE','Decision Tree - {}'.format(n_in)] = rmse(testy, yhat)
    
    # Random Forest
    gs_rf = GridSearchCV(estimator=RandomForestRegressor(random_state=0), cv=ts_splits,
                  param_grid={
                      'n_estimators': [10, 100, 1000],
                      'criterion':['mae'],
                      'max_depth':[10,100],
                      'min_samples_split': [5, 10, 100, 500],
                      'ccp_alpha': [0,0.001]
                  },
                  scoring='neg_mean_absolute_error',
                  verbose=0, n_jobs=-1)
    
    if force_retraining:
        gs_rf.fit(data_train[:, :-n_out], data_train[:, -n_out:].ravel())
        joblib.dump(gs_rf, 'models/Univariate/n_in_tuning/RandomForest_nin_{}.pkl'.format(n_in))
    else:
        gs_rf = joblib.load('models/Univariate/n_in_tuning/RandomForest_nin_{}.pkl'.format(n_in))
    print('RandomForest on {} tuned!'.format(n_in))    
    yhat = gs_rf.best_estimator_.predict(testX)
    output_metrics_preproc.loc['MAE','Random Forest - {}'.format(n_in)] = mean_absolute_error(testy, yhat)
    output_metrics_preproc.loc['RMSE','Random Forest - {}'.format(n_in)] = rmse(testy, yhat)
    
    # Gradient Boosting
    gs_gb = GridSearchCV(estimator=GradientBoostingRegressor(random_state=0),cv=ts_splits,
                  param_grid={
                      'loss': ['ls','lad'],
                      'n_estimators': [10, 100, 1000],
                      'criterion':['mse'],
                      'max_depth':[5,10,100],
                      'min_samples_split': [5, 10, 30],
                      'learning_rate': [0.000001, 0.01]
                  },
                  scoring='neg_mean_absolute_error',
                  verbose=0,n_jobs=-1)
    if force_retraining:
        gs_gb.fit(data_train[:, :-n_out], data_train[:, -n_out:].ravel())
        joblib.dump(gs_gb, 'models/Univariate/n_in_tuning/GradientBoosting_nin_{}.pkl'.format(n_in))
    else:
        gs_gb = joblib.load('models/Univariate/n_in_tuning/GradientBoosting_nin_{}.pkl'.format(n_in))
    print('GradientBoosting on {} tuned!'.format(n_in))   
    yhat = gs_gb.best_estimator_.predict(testX)
    output_metrics_preproc.loc['MAE','Gradient Boosting - {}'.format(n_in)] = mean_absolute_error(testy, yhat)
    output_metrics_preproc.loc['RMSE','Gradient Boosting - {}'.format(n_in)] = rmse(testy, yhat)
    
    # Super Learner
    reg1 = gs_dt.best_estimator_
    reg2 = gs_rf.best_estimator_
    reg3 = LinearRegression()
    ereg = VotingRegressor(estimators=[('dt', reg1), ('rf', reg2), ('lr', reg3)])
    if force_retraining:
        ereg = ereg.fit(data_train[:, :-n_out], data_train[:, -n_out:].ravel())
        joblib.dump(ereg, 'models/Univariate/n_in_tuning/SuperLearner_nin_{}.pkl'.format(n_in))
    else:
        ereg = joblib.load('models/Univariate/n_in_tuning/SuperLearner_nin_{}.pkl'.format(n_in))
    print('SuperLearner on {} tuned!'.format(n_in)) 
    yhat = ereg.predict(testX)
    output_metrics_preproc.loc['MAE','SuperLearner - {}'.format(n_in)] = mean_absolute_error(testy, yhat)
    output_metrics_preproc.loc['RMSE','SuperLearner - {}'.format(n_in)] = rmse(testy, yhat) 
    

In [ ]:
output_metrics_preproc.T.sort_values(by=['MAE','RMSE'])

In [ ]:
gs_dt = joblib.load('models/Univariate/n_in_tuning/DecisionTree_nin_24.pkl')
yhat = gs_dt.best_estimator_.predict(data.values[:,:-n_out])
plt.figure(figsize=(9,5))
plt.plot(data.values[:,-n_out:],'.-',label='Data')
plt.plot(yhat,'.-',label='Best Decision Tree')
plt.legend(frameon=False, loc=0)
plt.grid()

# 4 - Multivariate Analysis
This section will show the usage of CART models on a multivariate dataset. <br>
This dataset contains 19 different features such as air temperature, atmospheric pressure, and humidity collected from 2009 to 2016 with 1 records every hour. A complete <b>dataset analysis and feature engineering</b> can be found in the TensorFlow credits; for the sake of brevity, it will not be reported here. <br>
The goal is to <b>predict the air temperature</b> based on multiple features. Particularly interesting is the feature engineering on daily and yearly dependence:

- Two continuous variables (sine and cosine) will represent the time of the day and the time of the year. The usage of sine and cosine ensures unique values for both timings.


Credits: <br>https://www.bgc-jena.mpg.de/wetter/ <br>https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip <br>
https://www.tensorflow.org/tutorials/structured_data/time_series



In [ ]:
filename = 'weather_dataset.csv'
# load the dataset
df = pd.read_csv(filename, header=0, index_col=0, parse_dates=True)

# plot some features
feat = ['T (degC)','p (mbar)','Day sin', 'Day cos', 'Year sin','Year cos']
df.iloc[0:8761].plot(y=feat,style='-',grid=True,subplots=True, figsize=(8,6))
plt.show(block=False)
df.head()

<b>NOTE:</b> Before building any model, two naive models and a linear regression model are defined for performance comparison:
- The persistence model which simply predicts based on last seen record.
- The mean model which simply predicts the average of the training samples.
- Linear model: y = beta*X + alpha

<b>NOTE:</b> The dataset has been normalize with a standard scaler (subtract the mean and divide by the standard deviation). CART models do not need the scaling since the splits are calculated one feature at a time but in the next section Lasso regression will be used and it requires (or at least is good practice) to perform the scaling.

In [ ]:
############## Data Preprocessing Params ##############
n_in = 12 # Number of "previous data" to use as input
n_ahead = 6 # Number of step ahead to predict
single_output = True # If True, predicts only the last n_ahead value; else it predicts all the values "in between"
n_out = 1 if single_output else n_ahead # Number of outputs of the model
train_size = 0.7 # Size of the training set
n_splits = 5 # Number of splits for cross validation
features_tag = ['Day sin', 'Day cos', 'Year sin','Year cos','p (mbar)','Wx','Wy']
target_tag = 'T (degC)'
filename = 'weather_dataset.csv'
############## Data Preprocessing Params ##############

output_multi_metrics = pd.DataFrame(data={'Persistence':[float('nan'),float('nan')],
                                    'Mean':[float('nan'),float('nan')],
                                    'Linear':[float('nan'),float('nan')],
                                    'Lasso':[float('nan'),float('nan')],
                                    'DecisionTree':[float('nan'),float('nan')],
                                    'RandomForest':[float('nan'),float('nan')],
                                    'XGBoost':[float('nan'),float('nan')],
                                    'SuperLearner':[float('nan'),float('nan')]},index=['MAE','RMSE'])

# load the dataset
df = pd.read_csv(filename, header=0, index_col=0)

# Divide into training and test
train_size = int(train_size*df.shape[0])
df_train = df[features_tag].iloc[0:train_size]
df_test = df[features_tag].iloc[train_size:]

# Scale the data
train_mean, train_std = df_train.mean(), df_train.std()
df_train_normed = (df_train - train_mean) / train_std
df_test_normed = (df_test - train_mean) / train_std

df_train_normed[target_tag] = df.iloc[0:train_size][target_tag].values
df_test_normed[target_tag] = df.iloc[train_size:][target_tag].values

# Transform the time series data into supervised learning
data_train = series_to_supervised(df_train_normed.values, n_in=n_in, n_out=n_ahead, single_output=single_output)
data_train_target_only = series_to_supervised(df_train_normed[[target_tag]].values, n_in=n_in, n_out=n_ahead, single_output=single_output)
X_train, y_train = data_train.values[:,:-1], data_train.values[:,-1]
data_test = series_to_supervised(df_test_normed.values, n_in=n_in, n_out=n_ahead, single_output=single_output)
data_test_target_only = series_to_supervised(df_test_normed[[target_tag]].values, n_in=n_in, n_out=n_ahead, single_output=single_output)
X_test, y_test = data_test.values[:, :-1], data_test.values[:, -1]

print('Train size: {}'.format(data_train.shape))
print('Test size: {}'.format(data_test.shape))
df_train_normed.head()

### Important note:
The dimension of the dataset is <b>massive</b> so the training process is very slow; to mitigate this problem, it has been decided to use just 2 years of data for the training process. Another possible solution could have been to resample the data (for example: 1 point every 6 hours) but the goal (in this notebook) is to predict the temeprature 1 hour ahead.

In [ ]:
train_size = 2*365*24 # number of hours in 2 years
# Divide into training and test
df_train = df[features_tag].iloc[0:train_size]
df_test = df[features_tag].iloc[train_size:]

# Scale the data
train_mean, train_std = df_train.mean(), df_train.std()
df_train_normed = (df_train - train_mean) / train_std
df_test_normed = (df_test - train_mean) / train_std

df_train_normed[target_tag] = df.iloc[0:train_size][target_tag].values
df_test_normed[target_tag] = df.iloc[train_size:][target_tag].values

# Transform the time series data into supervised learning
data_train = series_to_supervised(df_train_normed.values, n_in=n_in, n_out=n_ahead, single_output=single_output)
data_train_target_only = series_to_supervised(df_train_normed[[target_tag]].values, n_in=n_in, n_out=n_ahead, single_output=single_output)
X_train, y_train = data_train.values[:,:-1], data_train.values[:,-1]
data_test = series_to_supervised(df_test_normed.values, n_in=n_in, n_out=n_ahead, single_output=single_output)
data_test_target_only = series_to_supervised(df_test_normed[[target_tag]].values, n_in=n_in, n_out=n_ahead, single_output=single_output)
X_test, y_test = data_test.values[:, :-1], data_test.values[:, -1]

print('Train size: {}'.format(data_train.shape))
print('Test size: {}'.format(data_test.shape))
df_train_normed.head()

As stated before, let's inspect the performances of 3 different techniques that will be used as a reference for performance.

In [ ]:
X_target_only, y_target_only = data_test_target_only.values[:, :-1], data_test_target_only.values[:, -1]

y_pers =  X_target_only[:,-1]
y_mean =  X_target_only.mean(axis=1)
reg = LinearRegression().fit(X_train, y_train)
y_lin = reg.predict(X_test)

n_days = 3

f, ax = plt.subplots(2,1,sharex=False, figsize=(9,6))
ax[0].plot(y_test[0:24*n_days],'.-',label='Temperature')
ax[0].plot(y_pers[0:24*n_days],'.-',label='Persistence Model', alpha = 0.5)
ax[0].plot(y_mean[0:24*n_days],'.-',label='Mean Model', alpha = 0.5)
ax[0].plot(y_lin[0:24*n_days], '.-',label='Linear Model', alpha = 0.5)
ax[0].set_title('Zoom on {} days'.format(n_days))
ax[0].legend(frameon=False)
ax[1].plot(y_test,'.-',label='Temperature')
ax[1].plot(y_pers,'.-',label='Persistence Model', alpha = 0.5)
ax[1].plot(y_mean,'.-',label='Mean Model', alpha = 0.5)
ax[1].plot(y_lin, '.-',label='Linear Model', alpha = 0.5)
ax[1].set_title('Entire dataset')
ax[0].set_ylabel('T [degC]')
ax[1].set_ylabel('T [degC]')
ax[1].set_xlabel('Time')
for a in ax: a.grid(True);

print('Persistence - test MAE = {:.3f}'.format(mean_absolute_error(y_test, y_pers)))
print('Mean - test MAE = {:.3f}'.format(mean_absolute_error(y_test, y_mean)))
print('Linear - test MAE = {:.3f}'.format(mean_absolute_error(y_test, y_lin)))
print('Persistence - test RMSE = {:.3f}'.format(rmse(y_test, y_pers)))
print('Mean - test RMSE = {:.3f}'.format(rmse(y_test, y_mean)))
print('Linear - test RMSE = {:.3f}'.format(rmse(y_test, y_lin)))
output_multi_metrics.loc['MAE','Persistence'] = mean_absolute_error(y_test, y_pers)
output_multi_metrics.loc['MAE','Mean'] = mean_absolute_error(y_test, y_mean)
output_multi_metrics.loc['MAE','Linear'] = mean_absolute_error(y_test, y_lin)
output_multi_metrics.loc['RMSE','Persistence'] = rmse(y_test, y_pers)
output_multi_metrics.loc['RMSE','Mean'] = rmse(y_test, y_mean)
output_multi_metrics.loc['RMSE','Linear'] = rmse(y_test, y_lin)

<b>NOTE:</b> it is clear from the very high MAE and RMSE (and from second subplots too) that linear regression is having a very unstable behavior in some areas.<br>
Let's inspect the coefficients of the regression.

In [ ]:
reg.coef_

It is evident that some coefficients have very high values; since just daily and yearly dependece is considered in this notebook, these coefficients have no reason to be so high.<br>
To mitigate this problem, let's apply the <b>Lasso</b> regularization technique and let's compare the coefficients.

In [ ]:
reg_lasso = Lasso(alpha=0.1, max_iter=2000).fit(X_train, y_train)
y_lasso = reg_lasso.predict(X_test)
output_multi_metrics.loc['MAE','Lasso'] = mean_absolute_error(y_test, y_lasso)
output_multi_metrics.loc['RMSE','Lasso'] = rmse(y_test, y_lasso)

n_days = 3

f, ax = plt.subplots(2,1,sharex=False, figsize=(9,6))
ax[0].plot(y_test[0:24*n_days],'.-',label='Temperature')
ax[0].plot(y_lin[0:24*n_days], '.-',label='Linear Model', alpha = 0.5)
ax[0].plot(y_lasso[0:24*n_days], '.-',label='Lasso Model', alpha = 0.5)
ax[0].set_title('Zoom on {} days'.format(n_days))
ax[0].legend(frameon=False)
ax[1].plot(y_test,'.-',label='Temperature')
# ax[1].plot(y_lin, '.-',label='Linear Model', alpha = 0.5)
ax[1].plot(y_lasso,label='Lasso Model')
ax[1].set_title('Entire dataset')
ax[0].set_ylabel('T [degC]')
ax[1].set_ylabel('T [degC]')
ax[1].set_xlabel('Time')
for a in ax: a.grid(True);

c = pd.DataFrame(data={'Linear coeff': reg.coef_,'Lasso coeff':reg_lasso.coef_}).sort_values(by='Linear coeff', ascending=False)
c.head(20)

In [ ]:
output_multi_metrics

## 4.1 Multivariate Analysis

### 4.1.1 Decision Tree - DA FINIRE
As done for the univariate case, a Decision Tree with hyper-parameter tuning has been trained.

In [ ]:
force_retraining = True
gs = GridSearchCV(estimator=DecisionTreeRegressor(random_state=0), cv=TimeSeriesSplit(n_splits=n_splits), 
                  param_grid={
                      'criterion':['mae'],
                      'max_depth':[5,50,300, 1000],
                      'min_samples_split': [10, 100],
                      'ccp_alpha': [0,0.001, 0.1, 1]        
                  },
                  scoring='neg_mean_absolute_error',
                  refit=True,
                  verbose = 10, n_jobs=-1)
if force_retraining:
    gs.fit(X_train, y_train)
    joblib.dump(gs, 'models/Multivariate/DecisionTree.pkl')
else:
    gs = joblib.load('models/Multivariate/DecisionTree.pkl')
print('Decision Tree - training MAE = {:.3f}'.format(abs(gs.cv_results_['mean_test_score'][0])))
print('Decision Tree - test MAE = {:.3f}'.format(mean_absolute_error(y_test, gs.best_estimator_.predict(X_test))))
print('Decision Tree - test RMSE = {:.3f}'.format(rmse(y_test, gs.best_estimator_.predict(X_test))))
gs_res = pd.DataFrame(gs.cv_results_).sort_values(by='rank_test_score')[['params','mean_test_score']]
gs_res.head(10)

The best model of the above list is the number 3, since it is the "simplest" one among the group with best performances.

In [ ]:
idx = 3
force_retraining = True
if force_retraining:
    dt_tuned_multi = DecisionTreeRegressor(random_state=0)
    dt_tuned_multi.set_params(**gs_res.loc[idx,'params'])
    dt_tuned_multi.fit(X_train, y_train)
    joblib.dump(dt_tuned_multi, 'models/Multivariate/DecisionTree_finalfit.pkl')
else:
    dt_tuned_multi = joblib.load('models/Multivariate/DecisionTree_finalfit.pkl')

In [ ]:
y_test_hat = dt_tuned_multi.predict(X_test)

def plot_multivariate(yhat, label, n_days=365):
    f, ax = plt.subplots(3,1,sharex=False, figsize=(9,6))
    ax[0].plot(y_test[0:24*n_days],'.-',label='Temperature')
    ax[0].plot(yhat[0:24*n_days], '.-',label=label, alpha = 0.5)
    ax[0].plot(y_lasso[0:24*n_days], '.-',label='Lasso Model', alpha = 0.5)
    ax[0].set_title('Zoom on {} days'.format(n_days))
    ax[0].legend(frameon=False)
    ax[1].plot(y_test,'.-',label='Temperature')
    ax[1].plot(yhat, '.-',label=label, alpha = 0.5)
    ax[1].plot(y_lasso,label='Lasso Model', alpha = 0.5)
    ax[1].set_title('Entire dataset')
    ax[2].plot(y_test - yhat, '.-',color='C1',label=label)
    ax[2].plot(y_test - y_lasso,color='C2',label='Lasso Model')
    ax[0].set_ylabel('T [degC]')
    ax[1].set_ylabel('T [degC]')
    ax[2].set_ylabel('Residuals')
    ax[2].set_xlabel('Time')
    ax[2]._shared_x_axes.join(ax[2],ax[1])
    for a in ax: a.grid(True);
plot_multivariate(y_test_hat, label='Decision Tree')
output_multi_metrics.loc['MAE','DecisionTree'] = mean_absolute_error(y_test, y_test_hat)
output_multi_metrics.loc['RMSE','DecisionTree'] = rmse(y_test, y_test_hat)

In [ ]:
output_multi_metrics

### 4.1.2 Random Forest - DA FINIRE
As done for the univariate case, a Random Forest regressor with hyper-parameter tuning has been trained.

In [ ]:
force_retraining = True
gs = GridSearchCV(estimator=RandomForestRegressor(random_state=0), cv=TimeSeriesSplit(n_splits=n_splits), 
                  param_grid={
                      'n_estimators': [10, 100, 1000],
                      'criterion':['mae'],
                      'max_depth':[100,500],
                      'max_features': ['auto', 'log2', 0.5, 0.75],
                      'min_samples_split': [10, 100]
                      'ccp_alpha': [0,0.001,0.1]
                  },
                  scoring='neg_mean_absolute_error',
                  verbose=10, n_jobs=-1)
if force_retraining:
    gs.fit(X_train, y_train)
    joblib.dump(gs, 'models/Multivariate/RandomForest.pkl')
else:
    gs = joblib.load('models/Multivariate/RandomForest.pkl')
print('RandomForest - training MAE = {:.3f}'.format(abs(gs.cv_results_['mean_test_score'][0])))
print('RandomForest - test MAE = {:.3f}'.format(mean_absolute_error(y_test, gs.best_estimator_.predict(X_test))))
print('RandomForest - test RMSE = {:.3f}'.format(rmse(y_test, gs.best_estimator_.predict(X_test))))
gs_res = pd.DataFrame(gs.cv_results_).sort_values(by='rank_test_score')[['params','mean_test_score']]
gs_res.head(10)

spiega

In [ ]:
idx = 3
force_retraining = True
if force_retraining:
    rf_tuned_multi = RandomForestRegressor(random_state=0, n_jobs=-1)
    rf_tuned_multi.set_params(**gs_res.loc[idx,'params'])
    rf_tuned_multi.fit(X_train, y_train)
    joblib.dump(rf_tuned_multi, 'models/Multivariate/RandomForest_finalfit.pkl')
else:
    rf_tuned_multi = joblib.load('models/Multivariate/RandomForest_finalfit.pkl')
y_test_hat = rf_tuned_multi.predict(X_test)
plot_multivariate(y_test_hat, label='Random Forest')
output_multi_metrics.loc['MAE','RandomForest'] = mean_absolute_error(y_test, y_test_hat)
output_multi_metrics.loc['RMSE','RandomForest'] = rmse(y_test, y_test_hat)

In [ ]:
output_multi_metrics

### 4.1.3 Extreme Gradient Boosting (xgboost) - DA FINIRE
In the univariate case, the GradientBoosting algorithm from Scikit Learn has been used. In the multivariate case the Extreme Gradient Boosting algorithm is going to be used because of its proven performances in training time. <br>
The main parameters are the following:
- "n_estimator": number of estimator to be trained by the algorithm.
- "max_depth": maximum depth of the tree. This parameter controls the maximum depth of the tree to avoid overfitting.
- "learning_rate": the learning rate multiplier for each additional tree.
- "objective": scoring metric to be used. Here instead of MAE, MSE will be used.
- "booster": Specify which booster to use: gbtree, gblinear or dart.
- "tree_method": euristics to increase training speed.
- "subsample": the fraction of samples to be used for fitting the individual base learners.
- "colsample_bytree": the number of features to consider when looking for the best split (smiliar to "max_features")

In [ ]:
force_retraining = True
gs = GridSearchCV(estimator=xgb.XGBRegressor(random_state=0), cv=TimeSeriesSplit(n_splits=n_splits), 
                  param_grid={
                      'n_estimators': [10,20],# [10, 100, 1000],
                      'max_depth':[10], #[100,500],                      
                      'learning_rate': [0.01], # [0.000001, 0.0001, 0.01, 0.1]
                      'booster': ['dart'], #['gbtree','gblinear']
                      'tree_method': ['approx','hist'], #['auto','approx'],
                      'subsample':[0.5], #[0.75,1]
                      'colsample_bytree': [0.5] #[0.5, 0.75, 0.95],
                  },
                  scoring='neg_mean_absolute_error',
                  verbose=10, n_jobs=-1)
if force_retraining:
    gs.fit(X_train, y_train)
    joblib.dump(gs, 'models/Multivariate/XGBoost.pkl')
else:
    gs = joblib.load('models/Multivariate/XGBoost.pkl')
print('XGBoost - training MAE = {:.3f}'.format(abs(gs.cv_results_['mean_test_score'][0])))
print('XGBoost - test MAE = {:.3f}'.format(mean_absolute_error(y_test, gs.best_estimator_.predict(X_test))))
print('XGBoost - test RMSE = {:.3f}'.format(rmse(y_test, gs.best_estimator_.predict(X_test))))
gs_res = pd.DataFrame(gs.cv_results_).sort_values(by='rank_test_score')[['params','mean_test_score']]
gs_res.head(10)

spiega

In [ ]:
idx = 3
force_retraining = True
if force_retraining:
    xgb_tuned_multi = xgb.XGBRegressor(random_state=0, n_jobs=-1)
    xgb_tuned_multi.set_params(**gs_res.loc[idx,'params'])
    xgb_tuned_multi.fit(X_train, y_train)
    joblib.dump(xgb_tuned_multi, 'models/Multivariate/XGBoost_finalfit.pkl')
else:
    xgb_tuned_multi = joblib.load('models/Multivariate/XGBoost_finalfit.pkl')
y_test_hat = xgb_tuned_multi.predict(X_test)
plot_multivariate(y_test_hat, label='XGB')
output_multi_metrics.loc['MAE','XGBoost'] = mean_absolute_error(y_test, y_test_hat)
output_multi_metrics.loc['RMSE','XGBoost'] = rmse(y_test, y_test_hat)

In [ ]:
output_multi_metrics

### 4.1.4 SuperLearner - DA FINIRE

In [ ]:
force_retraining = True
reg1 = dt_tuned_multi
reg2 = rf_tuned_multi
reg3 = Lasso(alpha=0.1, max_iter=2000).fit(X_train, y_train)
reg4 = xgb_tuned_multi
ereg = VotingRegressor(estimators=[('dt', reg1), ('rf', reg2), ('lr', reg3), ('xgb', reg4)])
if force_retraining:
    ereg = ereg.fit(X_train, y_train)
    joblib.dump(ereg, 'models/Multivariate/SuperLearner.pkl')
else:
    ereg = joblib.load('models/Multivariate/SuperLearner.pkl')

y_test_hat = ereg.predict(testX)
plot_multivariate(y_test_hat, label='XGB')
output_multi_metrics.loc['MAE','SuperLearner'] = mean_absolute_error(y_test, y_test_hat)
output_multi_metrics.loc['RMSE','SuperLearner'] = rmse(y_test, y_test_hat)

In [ ]:
output_multi_metrics

# 5 - Conclusions

In [ ]:
# ############## Params START ##############
# n_in = 6 # Number of "previous data" to use as input
# n_ahead = 1 # Number of step ahead to predict
# single_output = True # If True, predicts only the last n_ahead value; else it predicts all the values "in between"
# n_out = 1 if single_output else n_ahead # Number of outputs of the model
# train_size = 0.8 # Size of the training set
# n_splits = 5 # Number of splits for cross validation
# filename = 'sunspots.csv'
# ############## Params End ##############

# # load the dataset
# series = pd.read_csv(filename, header=0, index_col=0)

# # Transform the time series data into supervised learning
# data = series_to_supervised(series.values, n_in=n_in, n_out=n_ahead, single_output=single_output)
# data_train = data.values[0:int(train_size*data.shape[0])]
# data_test = data.values[int(train_size*data.shape[0]):]

# gs = GridSearchCV(estimator=model, cv=ts_splits, 
#                   param_grid={
#                       'criterion':['mae','mse'],
#                       'max_depth':[3],
#                       'min_samples_split': [5],
#                       'ccp_alpha': [0,1]                      
#                   },
#                   scoring='neg_mean_absolute_error')
# gs.fit(data_train[:, :-n_out], data_train[:, -n_out:])
# print('Decision Tree - training MAE = {:.3f}'.format(abs(gs.cv_results_['mean_test_score'][0])))
# print('Decision Tree - test MAE = {:.3f}'.format(mean_absolute_error(testy, gs.best_estimator_.predict(testX))))
# gs_res = pd.DataFrame(gs.cv_results_).sort_values(by='rank_test_score')[['params','mean_test_score']]
# gs_res.head(15)